In [ ]:
# This is the model in which we write the algorthim to decide how to allocate portfolio weights 
# This will ultimately output a dataset which all the user needs to do is just use the backtesting functionality
# that we end up writing out in a python notebook to get the plots that we need to analyze what we are working on
# class PortfolioModel:
#     def __init__(self, symbols: dict=Basket().get_portfolio()):
#         self.symbols = symbols
#         self.add = Ticker().recommend_tickers()
    
#     def add_tickers(self, tickers: list):
#         if tickers == self.add:

    
#     def get_tickers(self):
#         pass

In [20]:
import yfinance as yf

yf.download(tickers=['AXP','AAPL','JNJ','KO'],period="5y").dropna()



[*********************100%%**********************]  4 of 4 completed


Price        Adj Close                                          Close  \
Ticker            AAPL         AXP         JNJ         KO        AAPL   
Date                                                                    
2019-05-20   44.174908  111.725578  120.783432  41.820183   45.772499   
2019-05-21   45.021790  112.387520  120.521645  41.606163   46.650002   
2019-05-22   44.100113  111.884064  121.080124  42.505062   45.695000   
2019-05-23   43.347340  110.411041  121.978874  42.676277   44.915001   
2019-05-24   43.180859  111.417931  121.987686  42.470825   44.742500   
...                ...         ...         ...        ...         ...   
2024-05-13  186.279999  238.649994  151.220001  63.580002  186.279999   
2024-05-14  187.429993  241.529999  151.380005  63.099998  187.429993   
2024-05-15  189.720001  241.699997  152.669998  63.130001  189.720001   
2024-05-16  189.839996  241.320007  154.279999  63.320000  189.839996   
2024-05-17  189.869995  242.820007  154.639999  63.029999  189.869995   

Price                                                High              ...  \
Ticker             AXP         JNJ         KO        AAPL         AXP  ...   
Date                                                                   ...   
2019-05-20  119.839996  138.419998  48.849998   46.087502  120.320000  ...   
2019-05-21  120.550003  138.119995  48.599998   47.000000  120.680000  ...   
2019-05-22  120.010002  138.759995  49.650002   46.427502  120.879997  ...   
2019-05-23  118.430000  139.789993  49.849998   45.134998  119.400002  ...   
2019-05-24  119.510002  138.850006  49.610001   45.535000  119.629997  ...   
...                ...         ...        ...         ...         ...  ...   
2024-05-13  238.649994  151.220001  63.580002  187.100006  243.539993  ...   
2024-05-14  241.529999  151.380005  63.099998  188.300003  242.089996  ...   
2024-05-15  241.699997  152.669998  63.130001  190.649994  242.869995  ...   
2024-05-16  241.320007  154.279999  63.320000  191.100006  243.350006  ...   
2024-05-17  242.820007  154.639999  63.029999  190.809998  243.190002  ...   

Price              Low                   Open                          \
Ticker             JNJ         KO        AAPL         AXP         JNJ   
Date                                                                    
2019-05-20  137.679993  48.759998   45.880001  118.830002  138.610001   
2019-05-21  138.080002  48.450001   46.305000  120.470001  139.119995   
2019-05-22  137.490005  48.720001   46.165001  120.360001  138.350006   
2019-05-23  137.669998  49.459999   44.950001  119.269997  138.500000   
2019-05-24  138.559998  49.529999   45.049999  119.209999  139.500000   
...                ...        ...         ...         ...         ...   
2024-05-13  149.710007  63.180000  185.440002  242.949997  150.039993   
2024-05-14  150.699997  62.799999  187.509995  238.149994  151.850006   
2024-05-15  151.300003  63.000000  187.910004  241.630005  151.820007   
2024-05-16  152.330002  63.070000  190.470001  241.720001  153.000000   
2024-05-17  153.710007  62.939999  189.509995  242.800003  154.860001   

Price                     Volume                              
Ticker             KO       AAPL      AXP      JNJ        KO  
Date                                                          
2019-05-20  49.040001  154449200  2765600  4771100  13850500  
2019-05-21  49.000000  113459200  3369100  4859000  10587400  
2019-05-22  48.779999  118994400  2941100  5456800  16050400  
2019-05-23  49.500000  146118800  3290200  6951800  12199400  
2019-05-24  49.820000   94858800  2240100  5473300   6887600  
...               ...        ...      ...      ...       ...  
2024-05-13  63.240002   72044800  2768800  5628500  10137600  
2024-05-14  63.580002   52393600  2387300  8254200  10852200  
2024-05-15  63.060001   70400000  2013400  5548000   9838900  
2024-05-16  63.240002   52845200  2605900  7591000  10228000  
2024-05-17  63.340000   41260800  1848700 

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_torus(ticker,big,rolling_mean,rolling_std,buy_signal,sell_signal,size):
    # Define parametric equations for a torus
    def torus_parametric(u, v, R, r):
        x = (R + r * np.cos(v)) * np.cos(u)
        y = (R + r * np.cos(v)) * np.sin(u)
        z = r * np.sin(v)
        return x, y, z

    # Parameters for the torus
    R = (rolling_mean[ticker] + (2 * rolling_std[ticker])).mean()  # Major radius
    r = rolling_mean[ticker].mean()  # Minor radius

    # Generate torus coordinates
    u = np.linspace(0,big,size)
    v = np.linspace(0,big,size)
    U, V = np.meshgrid(u, v)
    X, Y, Z = torus_parametric(U, V, R, r)

    # Plot the torus
    fig = plt.figure(figsize=(20, 20))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, alpha=0.1, cmap='viridis')

    # Plot the price data
    ax.scatter(X[buy_signal[ticker]], Y[buy_signal[ticker]], Z[buy_signal[ticker]], color='g', label='Buy Signal')
    ax.scatter(X[sell_signal[ticker]], Y[sell_signal[ticker]], Z[sell_signal[ticker]], color='r', label='Sell Signal')
    ax.tick_params(labelsize=10)
    
    plt.show()

def action_tickers(ticker,buy_indicator,sell_indicator):
    tick_buy = pd.DataFrame(buy_indicator[ticker])
    tick_buy["BuySignal"] = "Buy"
    tick_buy = tick_buy.rename(columns={ticker:"BuyAction"})

    tick_sell = pd.DataFrame(sell_indicator[ticker])
    tick_sell["SellSignal"] = "Sell"
    tick_sell = tick_sell.rename(columns={ticker:"SellAction"})

    actions = pd.concat([tick_buy,tick_sell],axis=1)
                     
    actions.loc[(actions["BuyAction"] == False) & (actions["SellAction"] == False), ["BuySignal","SellSignal"]] = "Hold"

    buy = actions.loc[(actions["BuySignal"] == "Buy") & (actions["BuyAction"] == True), "BuySignal"]
    sell = actions.loc[(actions["SellSignal"] == "Sell") & (actions["SellAction"] == True), "SellSignal"]
    hold = actions.loc[(actions["BuySignal"] == "Hold") & (actions["SellSignal"] == "Hold"), "BuySignal"]

    actions = pd.concat([buy,sell,hold],axis=0).reset_index()

    actions.columns = ["Date", ticker + "Signal"]

    actions = actions.sort_values(by="Date",ascending=True).reset_index(drop=True)

    actions = actions.drop(["Date"],axis=1)

    return actions


big = 2 * np.pi
df_filter = (df - df.loc[df.index.date == start_date].values)

# Calculate the mean and standard deviation 

rolling_mean = df_filter.rolling(window=50).mean()
rolling_std = df_filter.rolling(window=50).std()

buy_signal = df_filter < rolling_mean - (2 * rolling_std)
sell_signal = df_filter > rolling_mean + (2 * rolling_std)

buy_signal = buy_signal.loc[buy_signal.index.date >= start_date]
sell_signal = sell_signal.loc[sell_signal.index.date >= start_date]

ticker_action = pd.DataFrame(buy_signal.index)

for tick in symbols:
    ticker_action[tick.lower() + "Signal"] = action_tickers(tick.lower(),buy_signal,sell_signal)

date = ticker_action.pop("Date")
ticker_action = ticker_action.set_index(date)

ticker_action

In [ ]:
plot_torus("cvx",big,rolling_mean.loc[rolling_mean.index >= start_date.strftime("%Y-%m-%d")],
           rolling_std.loc[rolling_std.index >= start_date.strftime("%Y-%m-%d")],buy_signal,sell_signal,len(ticker_action))

In [ ]:
def action_tickers(ticker,buy_indicator,sell_indicator):
    tick_buy = pd.DataFrame(buy_indicator[ticker])
    tick_buy["BuySignal"] = "Buy"
    tick_buy = tick_buy.rename(columns={ticker:"BuyAction"})

    tick_sell = pd.DataFrame(sell_indicator[ticker])
    tick_sell["SellSignal"] = "Sell"
    tick_sell = tick_sell.rename(columns={ticker:"SellAction"})

    actions = pd.concat([tick_buy,tick_sell],axis=1)
                     
    actions.loc[(actions["BuyAction"] == False) & (actions["SellAction"] == False), ["BuySignal","SellSignal"]] = "Hold"

    buy = actions.loc[(actions["BuySignal"] == "Buy") & (actions["BuyAction"] == True), "BuySignal"]
    sell = actions.loc[(actions["SellSignal"] == "Sell") & (actions["SellAction"] == True), "SellSignal"]
    hold = actions.loc[(actions["BuySignal"] == "Hold") & (actions["SellSignal"] == "Hold"), "BuySignal"]

    actions = pd.concat([buy,sell,hold],axis=0).reset_index()

    actions.columns = ["Date", ticker + "Signal"]

    actions = actions.sort_values(by="Date",ascending=True).reset_index(drop=True)

    actions = actions.drop(["Date"],axis=1)

    return actions

ticker_action = pd.DataFrame(df.index)

for tick in symbols:
    ticker_action[tick.lower() + "Signal"] = action_tickers(tick.lower(),buy_signal,sell_signal)

date = ticker_action.pop("Date")
ticker_action = ticker_action.set_index(date)

ticker_action

In [ ]:
import os
import subprocess
import bt
import datetime as dt
import numpy as np
import json

criteria = json.loads(open(os.getcwd() + "\\criteria.json","r").read())

tickers = subprocess.run(args=["python", os.getcwd() + "\\data_cleaning.py"],text=True,stdout=subprocess.PIPE)

results = tickers.stdout.strip('""[]\n').split(', ')

symbols = [element.strip("'\"") for element in results]

# current_date = dt.date.today()
# start_date = current_date - dt.timedelta(days=2000)

# num_day = start_date.isoweekday()

# if num_day > 5:
#     start_date = start_date - dt.timedelta(days=num_day-5)

# start = start_date - dt.timedelta(days=round((1.5*50)-1,0))

# if start.isoweekday() == 6:
#     start = start - dt.timedelta(days=1)
# elif start.isoweekday() == 7:
#     start = start + dt.timedelta(days=1)

df = bt.get(symbols,start="2014-01-01")

df_filter = df - df.iloc[0,:].values

norm_price = ((df_filter - df_filter.min()) / (df_filter.max()-df_filter.min())) * (2*np.pi)

rolling_mean = norm_price.rolling(window=50).mean()
rolling_std = norm_price.rolling(window=50).std()

buy_signal = norm_price < rolling_mean - (2 * rolling_std)
sell_signal = norm_price > rolling_mean + (2 * rolling_std)

# bt.algos.WeighSpecified(**weights)


strat = bt.Strategy('tailAction', [bt.algos.SelectWhere(sell_signal),
                                   bt.algos.WeighEqually(),
                                   bt.algos.Rebalance()])

# now we create the Backtest
t = bt.Backtest(strat, df.iloc[51:,:],integer_positions=False, progress_bar=True)

# and let's run it!
res = bt.run(t)

res.plot()


In [ ]:
import bt


strat = bt.Strategy('tailAction', [bt.algos.SelectWhere(buy_signal),
                                   bt.algos.WeighEqually(),
                                   bt.algos.Rebalance()])

# now we create the Backtest
t = bt.Backtest(strat, df.iloc[51:,:],integer_positions=False, progress_bar=True)

# and let's run it!
res = bt.run(t)

res.plot()

In [ ]:
import bt

strat = bt.Strategy('tailAction', [bt.algos.SelectWhere(sell_signal),
                                   bt.algos.SelectWhere(buy_signal),
                                   bt.algos.WeighEqually(),
                                   bt.algos.Rebalance()])

# now we create the Backtest
t = bt.Backtest(strat, df,integer_positions=False, progress_bar=True)

# and let's run it!
res = bt.run(t)

res.plot()